## Import

In [ ]:
!pip install zeep

In [ ]:
!pip install xmltodict

In [ ]:
from http import client # https://docs.python.org/3/library/http.client.html
import requests # https://pypi.org/project/requests/
import zeep # https://docs.python-zeep.org/en/master/
from urllib.request import urlopen # https://docs.python.org/3/library/urllib.html
from zeep import Client, Transport
import json
import xmltodict
from lxml import etree, html
import ssl
from requests.structures import CaseInsensitiveDict
import warnings
warnings.filterwarnings("ignore") # ssl verification false yapacağımz için sürekli uyarı verecek, bu olmadan da kod çalışır ama terminalde sürekli uyarı mesajı gözükür.

# istek atma yöntemleri ve verification=false yapma



In [ ]:
# urllib kütüphanesi ile request ederken --> urllib.request.urlopen(HOSTNAME, context = ssl._create_unverified_context())
# http client ile request ederken        --> http.client.HTTPSConnection(HOSTNAME, context = ssl._create_unverified_context())
# lxml.requests ile request ederken      --> requests.get(HOSTNAME, verify=False) 
# urllib.request ile request ederken     --> ctx = ssl.create_default_context()
                                           # ctx.check_hostname = False
                                           # ctx.verify_mode = ssl.CERT_NONE

                                           # with urllib.request.urlopen(url_string, context=ctx) as u, open(file_name, 'wb') as f:
                                                    # f.write(u.read())

# XPath alma

###***element xpath kopyalama:***


1.   sayfaya git
2.   sağ tıkla - inspect (F12)
3.   açılan DevTools penceresinde elements kısmına gel (zaten ilk açılan kısım)
4.   sol üstte ok işaretini kullanarak sayfada istediğin kısmı seç YA DA ctrl+f ile istediğin elemanı arat (örn: İletişim diye aratırsan o sayfada iletişim yazan her elemente gidersin)
5.  elementin üstüne sağ tıkla - copy XPath ya da copy full XPath

# CURL kopyalama ve CURL ile istek atma, koda çevirme

###***curl kopyalama:***
1. sayfaya git
2. sağ tıkla - inspect (F12)
3. sayfayı yenile
4. F12 ile açılan DevTools penceresinde network kısmına gel
5. listelenen .html, .js, .com...... gibi sayfalardan herhangi birinin üstüne gelip sağ tıkla - copy - copy as curl(cmd) seç
6. bütün ^ sil ve yeni metni kopyala
7. DevToolsta aşağıda console kısmına gelip kopyalanmış metni yapıştır, hepsini seç, tekrar kopyala
8. cmd açıp cd ile LIBCURL.EXE'nin olduğu dizine gir (\Downloads\curl-7.64.1\I386) --> https://skanthak.homepage.t-online.de/curl.html#download
9. curl metnini yapıştır
10. https://reqbin.com/curl sitesinde yapıştır ve generate code ile python kodu oluştur

# http client için metod


In [ ]:
def req_url(con, method, body): # con -> host adı
    connection = client.HTTPSConnection(con, context=ssl._create_unverified_context()) # ssl hatası almamak için verification false olmalı, http client kullanırken context = ssl._create_unverified_context() şeklinde parametre yollanır
    connection.request(method, body) # method -> request ederken  kullanılan yöntem (GET, POST, PUT... vs)
    return connection.getresponse()

# http client ile istek atma

In [ ]:
# http client kullanarak bursa burulaş'tan otobüs hatlarını indirme -> http://petekapi.burulas.com.tr/burulasweb/otobus/hatlar -> json
def bursa():
  response_bursa = req_url('petekapi.burulas.com.tr', 'GET', '/burulasweb/otobus/hatlar')
  json_response_bursa = response_bursa.read().decode('utf8')
  data_bursa = json.loads(json_response_bursa)
  for d in data_bursa:
    print(d["HatAdi"])
  # print(data_bursa)

# lxml.requests ile istek atma (get)

In [ ]:
  response_sakarya = requests.get('https://sakus.sakarya.bel.tr/', verify=False) 
  data_sakarya = response_sakarya.text
  doc_sakarya = html.fromstring(data_sakarya)
  html_element = doc_sakarya.xpath('/html/body')[0]
  print(html_element)
  print(html_element.get('onload'))

<Element body at 0x7f164de54410>
init()


In [ ]:
# lxml.requests kullanarak sakarya sakus'tan otobüs hatlarını indirme -> https://sakus.sakarya.bel.tr/ -> html 
def sakarya():
  response_sakarya = requests.get('https://sakus.sakarya.bel.tr/', verify=False) 
  data_sakarya = response_sakarya.text
  doc_sakarya = html.fromstring(data_sakarya)
  # print(data_sakarya)
  hatlar = doc_sakarya.xpath('//*[@id="drpHat"]//option')
  # print (hatlar)
  for hat in hatlar:
      # print(hat)
      hatid = hat.get('value') # bir html elementinden 'value' öğesini alma -> <option value="-100">Hat Seçiniz!</option> -> 'hat' option elementi. eğer <a src="www.google.com"> GOOGLE </a> elementinden 'href'in içeriğini almak isteseydik hat.get('href')
      hatname = hat.xpath('.//text()')[0] # html elementinin içindeki texti alma. sonuç liste olarak geldiği için listenin elemanlarından ihtiyacımız olanı seçmek gerekiyor
      print(hatid + '|' + hatname)

# requests.post ile CURL'den oluşturulmuş verilerle istek atma

In [ ]:
# requests.post ile malatya motaş'tan kart dolum bayilerinin adreslerini indirme -> https://www.motas.com.tr/ajax/card-store-point.php -> html
def malatya():
  response_malatya = req_url('motas.com.tr', 'GET', '/card/motas-kart-satis-dolum-noktalari.html') 
  page_response = response_malatya.read().decode('utf8')
  doc_malatya = html.fromstring(page_response)
  bayiler_malatya = doc_malatya.xpath('//*[@id="storeSelect"]/option')


  for bayi in bayiler_malatya: 
    if bayi.get('value') is not None:
        bayi_val = bayi.get('value')
        bayi_adi = bayi.xpath('.//text()')[0]
        print(bayi_val, bayi_adi)
        url = "https://www.motas.com.tr/ajax/card-store-point.php" # erişilecek 2. url

        # sayfaya post isteği atabilmek için gerekli diğer öğeler -> https://reqbin.com/curl sayfasından kopyalanan curl koda dönüştürülebilir. aynı şekilde cmd'de "curl-7.64.1\I386" dizinine gidip curl kodunu oraya yapıştırıp çalışıp çalışmadığı kontrol edilebilir.
        headers = CaseInsensitiveDict()
        headers["accept"] = "*/*"
        headers["content-type"] = "application/x-www-form-urlencoded; charset=UTF-8"
        headers["origin"] = "https://www.motas.com.tr"
        headers["referer"] = "https://www.motas.com.tr/card/motas-kart-satis-dolum-noktalari.html"
        headers["user-agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
        headers["x-requested-with"] = "XMLHttpRequest"

        # önceki sayfanın içinden bulduğumuz bayi_val ile diğer sayfaya post isteği atılıyor
        data = "valueSelected=" + bayi_val + "&is_ajax=1"

        response_malatya2 = requests.post(url, headers=headers, data=data, verify=False) # post işleminde de verify=False
        
        doc_malatya2 = html.fromstring(response_malatya2.text)
        adres = doc_malatya2.xpath('.//*[@class="card-body"]//*[@class="row"][3]//text()')
        adres_str = ""
        for a in adres:
          adres_str += a.replace("\n", " ").replace("\t", "") + " "
        print(adres_str)

# zeep ile wsdl'ye istek atıp servisleri kullanma

In [ ]:
# zeep ile iett api'den duyurları indirme
# api dokümanı -> https://data.ibb.gov.tr/dataset/53b985b6-24af-4fda-aa59-1b45dde2e665/resource/6efd7520-0fbf-421b-975a-a73cb9137ef2/download/iett-web-servis-kullanm-dokumanv.1.1.pdf
def istanbul():
    session = requests.Session()
    session.verify = False # ssl verification false
    transport = Transport(session=session)

    client = Client(wsdl="https://api.ibb.gov.tr/iett/FiloDurum/SeferGerceklesme.asmx?wsdl", transport=transport)

    data_text = client.service.GetFiloAracKonum_json() # gelen metodlardan birisine istek atılması
    # your_pretty_xml = etree.tostring(data_text, encoding="unicode", pretty_print=True) # etree ile treeye xml olarak çevrilmesi
    print(data_text) # gelen xml dosyasının yazılması
    # for d in list(data_text):
        #hatkodu = d.find('HAT').text
        #mesaj = d.find('MESAJ').text
        # print(hatkodu + "\n --->" + mesaj + "\n")







    # bu döngüde yukarıdaki linkteki web servisinde kullanılabilecek metodlar ve bu metodlara girilecek parametreler kaydediliyor
    interface = {}
    # e = client.service
    for service in client.wsdl.services.values():
        interface[service.name] = {}
        for port in service.ports.values():
            interface[service.name][port.name] = {}
            operations = {}
            for operation in port.binding._operations.values():
                operations[operation.name] = {}
                operations[operation.name]['input'] = {}
                elements = operation.input.body.type.elements
                operations[operation.name]['input'] = parseElements(elements)
            interface[servizce.name][port.name]['operations'] = operations
    print(interface) # metod parametre listesinin yazılması



def parseElements(elements):
    all_elements = {}
    for name, element in elements:
        all_elements[name] = {}
        all_elements[name]['optional'] = element.is_optional
        if hasattr(element.type, 'elements'):
            all_elements[name]['type'] = parseElements(
                element.type.elements)
        else:
            all_elements[name]['type'] = str(element.type)

    return all_elements


# urlopen ile internetteki bir xml dosyasını okuma

In [ ]:
def denizli():
    file = urlopen('https://ulasim.denizli.bel.tr/veri.ashx?t=belediye', context=ssl.SSLContext())
    data = file.read()
    file.close()
    data = xmltodict.parse(data)

    for belediye in data["Belediyeler"]["Belediye"]:
        print(belediye["ID"] + "|" + belediye["ADI"])

# metodları çağırma

In [ ]:
istanbul()

[{"Operator":"OHO","Garaj":null,"KapiNo":"A-001","Saat":"12:10:49","Boylam":"29.050858","Enlem":"41.088184","Hiz":"0","Plaka":"34 DTR 928"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-002","Saat":"12:10:53","Boylam":"29.051008","Enlem":"41.088474","Hiz":"0","Plaka":" 34 YM 3835"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-003","Saat":"12:10:50","Boylam":"29.050558","Enlem":"41.088123","Hiz":"0","Plaka":"34 TM 2315"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-004","Saat":"12:10:52","Boylam":"29.013521","Enlem":"41.066814","Hiz":"23","Plaka":"34 ZC 7661"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-005","Saat":"12:10:50","Boylam":"29.033049","Enlem":"41.051613","Hiz":"58","Plaka":"34 GDR 01"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-006","Saat":"12:10:49","Boylam":"29.002857","Enlem":"41.04132","Hiz":"0","Plaka":"34 AEK 961"},{"Operator":"OHO","Garaj":null,"KapiNo":"A-007","Saat":"12:10:48","Boylam":"28.992193","Enlem":"41.037254","Hiz":"27","Plaka":"34 NZ 0708"},{"Operator":"OHO","Garaj